In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

from lion_pytorch import Lion

In [2]:
# just workspace if you are using the docker container

directory_path = '/mnt/c/Users/fabi8/Desktop/GNN'

In [3]:
# one_hot_allNaNs_agep: alpha = 0.122
# one_hot_allNaNs_schl: alpha = 0.123
# one_hot_pNaNs_agep: alpha = 0.125
# one_hot_pNaNs_schl: alpha = 0.125

In [5]:
# test run 
alpha = .125
data_set = 'one_hot_pNaNs_agep_small.csv'
intermediate_models = 'best_model_small.pth'
output_model = 'model_small.pth'

In [5]:
# run 1
#data_set = 'one_hot_allNaNs_agep.csv'
#intermediate_models = 'best_model_allNaNs_agep.pth'
#output_model = 'model_allNaNs_agep.pth'
#alpha = .122

In [6]:
# run 2
#data_set = 'one_hot_allNaNs_schl.csv'
#intermediate_models = 'best_model_allNaNs_schl.pth'
#output_model = 'model_allNaNs_schl.pth'
#alpha = 0.123

In [ ]:
# run 3
#data_set = 'one_hot_pNaNs_agep.csv'
#intermediate_models = 'best_model_pNaNs_agep.pth'
#output_model = 'model_pNaNs_agep.pth'
#alpha = 0.125

In [8]:
# run 4
# data_set = 'one_hot_pNaNs_schl.csv'
# intermediate_models = 'best_model_pNaNs_schl.pth'
# output_model = 'model_pNaNs_schl.pth'
# alpha = 0.125

In [6]:
from vae import VAE
from vae import GroupSoftmax
from trainer import Trainer

In [7]:
# Device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [ ]:
# input size not updated
group_sizes = [2, 5, 5, 11, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
test = VAE(433, 1500, 6, 500, group_sizes)

total_params = sum(p.numel() for p in test.parameters())
trainable_params = sum(p.numel() for p in test.parameters() if p.requires_grad)
print(trainable_params)


57670299


In [ ]:
# Dataset import 
# Windows path
# data = pd.read_csv('A:/csv_hus/psam_hus_pus_filtered.csv')
# WSL path
data = pd.read_csv(f'{directory_path}/data/{data_set}')
data.head()

,TEN:owned or mortgaged,TEN:rented,HHL:english,HHL:spanish,HHL:other indo-european,HHL:asian and pacific island languages,HHL:other,VEH:no vehicles,VEH:1 vehicle,VEH:2 vehicles,...,SCHL_13:bachelor,SCHL_13:graduate,SCHL_13:nan,SCHL_14:other,SCHL_14:less than high school,SCHL_14:high school,SCHL_14:college or associate,SCHL_14:bachelor,SCHL_14:graduate,SCHL_14:nan
0,1,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
1,1,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2,1,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
3,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1


In [9]:
cols = list(data.columns)
cols = [col.split(":")[0] for col in cols]

onehot_counts = {col: sum(data.columns.str.startswith(f"{col}:")) for col in cols}
print(list(onehot_counts.values()))


input_dim = sum(onehot_counts.values())
print(input_dim)

[2, 5, 5, 11, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
433


In [10]:
# Data loader 

data_tensor = torch.tensor(data.values, dtype=torch.float32)
data_loader = DataLoader(data_tensor, batch_size=len(data_tensor), shuffle=True)

# Class_sizes 
group_sizes = list(onehot_counts.values())


In [16]:
model = VAE(input_dim, 1000, 6, 500, group_sizes)
# load pretrained model 
#params = torch.load('/workspace/models/model.pth')
#
#model.load_state_dict(params)

In [17]:
# optimizer
optimizer = Lion(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, device)

In [ ]:
# train the model 
model_path = f'{directory_path}/models/{intermediate_models}'
trainer.train(data_loader, 4000, model_path, gamma=2, alpha=alpha, beta=5)

Epoch 0, Loss: 42.71170425415039, LR: 0.001
Epoch 1, Loss: 25.971181869506836, LR: 0.001
Epoch 2, Loss: 22.62834930419922, LR: 0.001
Epoch 3, Loss: 3.2825634479522705, LR: 0.001
Epoch 4, Loss: 3.1712584495544434, LR: 0.001
Epoch 5, Loss: 3.1248228549957275, LR: 0.001
Epoch 6, Loss: 3.0979483127593994, LR: 0.001
Epoch 7, Loss: 3.082977771759033, LR: 0.001
Epoch 8, Loss: 3.067559242248535, LR: 0.001
Epoch 9, Loss: 3.0579328536987305, LR: 0.001
Epoch 10, Loss: 3.0500898361206055, LR: 0.001
Epoch 11, Loss: 3.0474977493286133, LR: 0.001
Epoch 12, Loss: 3.0496227741241455, LR: 0.001
Epoch 13, Loss: 3.053840398788452, LR: 0.001
Epoch 14, Loss: 3.0592801570892334, LR: 0.001
Epoch 15, Loss: 3.0644564628601074, LR: 0.001
Epoch 16, Loss: 3.0671961307525635, LR: 0.001
Epoch 17, Loss: 3.0665886402130127, LR: 0.001
Epoch 18, Loss: 3.062835216522217, LR: 0.001
Epoch 19, Loss: 3.0574142932891846, LR: 0.001
Epoch 20, Loss: 3.0490169525146484, LR: 0.001
Epoch 21, Loss: 3.038569688796997, LR: 0.001
Epoch

In [ ]:
torch.save(model.state_dict(), f'{directory_path}/models/{output_model}')